In [2]:
import os
import re
import sys
import time
import logging
import pandas as pd
import openpyxl

# =====================
# Configurations
# =====================
TEMPLATE_FILE_PATH = os.path.abspath("workday_data.xlsx")
OUTPUT_MASTER_FILE = "master_data.xlsx"
HR_OUTPUT_DIR = "hr_files"
LOGS_DIR = "logs"

MASTER_SHEET = "Masterdata_PSteam"
EMPLOYEE_MASTER_SHEET = "Employee Master"
HIRING_SHEET = "Hiring_data"
LINEUP_SHEET = "LINEUP_PASS_IMPORT"
# TERMINATION_SHEET = "Termination"  # Currently unused
PERMISSION_SHEET = "Permission"

SKIPROWS = {
    MASTER_SHEET: 11,
    EMPLOYEE_MASTER_SHEET: 11,
    HIRING_SHEET: 1,
    LINEUP_SHEET: 0,
    PERMISSION_SHEET: 0,
}

# Only append existing Masterdata_PSteam content if required
APPEND_EXISTING_MASTER = False

# Column mapping from source columns -> target schema columns
COLUMN_MAPPING = {
    'Emp id': 'EID',
    'Hire Date (DD-MMM-YYYY)': 'DOJ',
    'ORIGINAL_DATE_OF_HIRE': 'DOJ',
    'FULL_NAME': 'EMPLOYEE FULL NAME_EN',
    'Họ tên': 'EMPLOYEE FULL NAME_VN',
    'Gender': 'GENDER',
    'Giới tính': 'GENDER',
    'DOB (DD-MMM-YYYY)': 'DOB',
    'Dân tộc': 'ETHNIC',
    'Nơi sinh': 'BIRTH PLACE',
    'Số CMND/CCCD:': 'ID CARD',
    'Ngày cấp CMND/CCCD': 'ISSUED DATE',
    'Nơi cấp CCCD/CMND': 'ISSUED PLACE',
    'Số CMND Cũ (Nếu Có)': 'OLD ID CARD NO.',
    'Địa chỉ thường trú:': 'PERMANENT ADDRESS',
    'Địa chỉ tạm trú': 'TEMPORARY ADDRESS',
    'Highest Education': 'EDUCATION',
    'Tình trạng hôn nhân': 'MARITAL STATUS',
    'Phone Number': 'MOBILE PHONE NO',
    'Email - Work': 'CNX EMAIL',
    'Home E-mail address': 'PERSONAL EMAIL',
    'Số điện thoại người liên hệ trong trường hợp khẩn cấp': 'EMERGENCY CONTACT',
    'Số Bảo Hiểm Xã Hội': 'SOCIAL INSURANCE NUMBER',
    'Mã số thuế': 'PERSONAL TAX CODE',
    'Số tài khoản ngân hàng nhận lương': 'BANK ACCOUNT',
    'Tên ngân hàng': 'BANK NAME',
    'Base Pay': 'BASE SALARY',
    'Complexity Allowance': 'COMPLEXITY ALLOWANCE',  # fixed typo
    'Position Allowance': 'POSITION ALLOWANCE',
    'Meal Allowance - Monthly': 'MEAL ALLOWANCE',
    'Business Title': 'BUSINESS_TITLE_EN',
    'Vị trí': 'BUSINESS_TITLE_VN',
    'SUPERVISOR_FULL_NAME': 'SUPERVISOR',
    'Career Level': 'CAREER LEVEL',
    'Họ và tên chủ hộ': 'Họ tên chủ hộ',
    'Giới tính chủ hộ': 'Giới tính chủ hộ',
    'Ngày sinh của chủ hộ': 'DOB chủ hộ',
    'Mối quan hệ giữa chủ hộ và bạn': 'Mối quan hệ giữa NLĐ với chủ hộ',
    'MSA Client': 'MSA Client',
    'Contract ID': 'PROBATION CONTRACT NO',
    'Process': 'PROJECT',
    'Start Date (DD-MMM-YYYY)': 'FROM',
    'End Date (DD-MMM-YYYY)': 'TO',
    'Bệnh viện muốn đăng ký BHYT': 'HOSPITAL NAME',  # trim trailing space
    'Legislation Code / Country': 'NATIONALITY',
}

# =====================
# Logging
# =====================
os.makedirs(LOGS_DIR, exist_ok=True)
logging.basicConfig(
    filename=os.path.join(LOGS_DIR, 'merge_log.txt'),
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(message)s'
)


def log_and_print(message):
    logging.info(message)
    print(message)


# =====================
# Utility functions
# =====================

def read_sheet(path, sheet_name, skiprows=0, usecols=None):
    return pd.read_excel(
        path,
        engine='openpyxl',
        dtype=object,
        sheet_name=sheet_name,
        skiprows=skiprows,
        usecols=usecols
    )


def get_master_schema_columns(path):
    # Read header only to get schema columns, avoid loading entire sheet
    df_header = pd.read_excel(
        path,
        engine='openpyxl',
        sheet_name=MASTER_SHEET,
        skiprows=SKIPROWS[MASTER_SHEET],
        nrows=0
    )
    schema_cols = [col for col in df_header.columns if not str(col).startswith('Unnamed')]
    return schema_cols


def normalize_emp_id(series):
    if series is None:
        return series
    return series.astype(str).str.strip()


def normalize_national_id(series):
    if series is None:
        return series
    # keep digits only for national id, remove spaces and punctuation
    return series.astype(str).str.replace(r'\D', '', regex=True)


def safe_merge(left, right, left_on, right_on, how='left', validate=None, label=""):
    # Optional validation of key uniqueness
    if validate is not None:
        try:
            return left.merge(right, left_on=left_on, right_on=right_on, how=how, validate=validate)
        except Exception as exc:
            log_and_print(f"Merge validation failed for {label}: {exc}")
            return left.merge(right, left_on=left_on, right_on=right_on, how=how)
    return left.merge(right, left_on=left_on, right_on=right_on, how=how)


# =====================
# Main pipeline
# =====================
start_ts = time.time()
try:
    # 1) Read minimal inputs
    t0 = time.time()
    master_schema_cols = get_master_schema_columns(TEMPLATE_FILE_PATH)
    hiring_df = read_sheet(TEMPLATE_FILE_PATH, HIRING_SHEET, SKIPROWS[HIRING_SHEET])
    emp_master_df = read_sheet(TEMPLATE_FILE_PATH, EMPLOYEE_MASTER_SHEET, SKIPROWS[EMPLOYEE_MASTER_SHEET])
    lineup_df = read_sheet(TEMPLATE_FILE_PATH, LINEUP_SHEET, SKIPROWS[LINEUP_SHEET])
    permission_df = read_sheet(TEMPLATE_FILE_PATH, PERMISSION_SHEET, SKIPROWS[PERMISSION_SHEET])

    read_time = time.time() - t0

    # 2) Normalize join keys
    if 'Emp id' in hiring_df.columns:
        hiring_df['Emp id'] = normalize_emp_id(hiring_df['Emp id'])
    if 'EMPLOYEE_NUMBER' in emp_master_df.columns:
        emp_master_df['EMPLOYEE_NUMBER'] = normalize_emp_id(emp_master_df['EMPLOYEE_NUMBER'])

    if 'National ID (SSN/SIN) (National Identifiers)' in hiring_df.columns:
        hiring_df['National ID (SSN/SIN) (National Identifiers)'] = normalize_national_id(
            hiring_df['National ID (SSN/SIN) (National Identifiers)']
        )
    if 'Số CMND/CCCD:' in lineup_df.columns:
        lineup_df['Số CMND/CCCD:'] = normalize_national_id(lineup_df['Số CMND/CCCD:'])

    # 2.1) Deduplicate keys prior to merge
    def safe_filename(text):
        return re.sub(r'[<>:"/\\|?*]+', '_', str(text))

    def report_dups(df, key, name):
        if key in df.columns:
            dup = df[df.duplicated(subset=[key], keep=False) & df[key].notna()]
            if not dup.empty:
                log_and_print(f"Duplicate key detected in {name} on '{key}': {dup.shape[0]} rows across {dup[key].nunique()} keys")
                dups_dir = os.path.join(LOGS_DIR, 'dups')
                os.makedirs(dups_dir, exist_ok=True)
                out = os.path.join(dups_dir, f"dups_{safe_filename(name)}_{safe_filename(key)}.csv")
                dup.to_csv(out, index=False)
                # Keep the first occurrence by default
                df.drop_duplicates(subset=[key], keep='first', inplace=True)
        return df

    hiring_df = report_dups(hiring_df, 'Emp id', 'hiring')
    emp_master_df = report_dups(emp_master_df, 'EMPLOYEE_NUMBER', 'employee_master')
    lineup_df = report_dups(lineup_df, 'Số CMND/CCCD:', 'lineup')

    # 3) Merge hiring with employee master
    t1 = time.time()
    merged_first = safe_merge(
        hiring_df,
        emp_master_df,
        left_on='Emp id',
        right_on='EMPLOYEE_NUMBER',
        how='left',
        validate='many_to_one',  # many hires to one employee master row
        label='hiring_emp_master'
    )
    # remove suffixes
    merged_first = merged_first.loc[:, ~merged_first.columns.str.endswith('_y')]
    merged_first.columns = [c.replace('_x', '') for c in merged_first.columns]

    # 4) Merge with lineup by national id
    merged_second = safe_merge(
        merged_first,
        lineup_df,
        left_on='National ID (SSN/SIN) (National Identifiers)',
        right_on='Số CMND/CCCD:',
        how='left',
        validate='many_to_one',  # many hires to one lineup record
        label='with_lineup'
    )
    merged_second = merged_second.loc[:, ~merged_second.columns.str.endswith('_y')]
    merged_second.columns = [c.replace('_x', '') for c in merged_second.columns]

    merge_time = time.time() - t1

    # 5) Map columns to target schema
    t2 = time.time()
    merged_second = merged_second.dropna(how='all')

    # Build mapped dataframe according to schema
    mapped = {}
    for source_col, target_col in COLUMN_MAPPING.items():
        if source_col in merged_second.columns:
            mapped[target_col] = merged_second[source_col]
    mapped_df = pd.concat(mapped, axis=1) if mapped else pd.DataFrame(columns=master_schema_cols)

    # Ensure all schema columns exist and are ordered
    for col in master_schema_cols:
        if col not in mapped_df.columns:
            mapped_df[col] = pd.NA

    # Convert DOJ if it looks like an Excel serial number
    if 'DOJ' in mapped_df.columns:
        ser = mapped_df['DOJ']
        try:
            # try numeric conversion; values that fail remain NaN
            ser_num = pd.to_numeric(ser, errors='coerce')
            # Heuristic: Excel serials are typically > 20000
            mask_serial = ser_num.notna() & (ser_num > 20000)
            mapped_df.loc[mask_serial, 'DOJ'] = pd.to_datetime(ser_num[mask_serial], unit='D', origin='1899-12-30')
        except Exception:
            pass

    mapped_df = mapped_df[master_schema_cols]

    # Optionally append existing master rows (expensive)
    if APPEND_EXISTING_MASTER:
        master_existing = read_sheet(TEMPLATE_FILE_PATH, MASTER_SHEET, SKIPROWS[MASTER_SHEET])
        master_existing = master_existing.loc[:, ~master_existing.columns.str.startswith('Unnamed')]
        # Do not drop all NA rows globally to avoid losing partial records
        master_df = pd.concat([master_existing, mapped_df], ignore_index=True)
    else:
        master_df = mapped_df.copy()

    map_time = time.time() - t2

    # 6) Enrich with permission on "MSA Client"
    t3 = time.time()
    if 'MSA Client' in master_df.columns and 'MSA Client' in permission_df.columns:
        final_df = master_df.merge(permission_df, on='MSA Client', how='left')
    else:
        final_df = master_df
        log_and_print("Warning: 'MSA Client' not found in one of the datasets; skipping permission merge.")

    enrich_time = time.time() - t3

    # 7) Write master output using fast writer
    t4 = time.time()
    with pd.ExcelWriter(OUTPUT_MASTER_FILE, engine='xlsxwriter') as writer:
        final_df.to_excel(writer, sheet_name='master_data', index=False)
    write_master_time = time.time() - t4

    # 8) Export HR files by SSO (skip NaN SSO)
    t5 = time.time()
    if 'SSO' not in final_df.columns:
        raise ValueError("Cột 'SSO' không tồn tại trong master_data")

    os.makedirs(HR_OUTPUT_DIR, exist_ok=True)
    groups = final_df.dropna(subset=['SSO']).groupby('SSO', dropna=True)
    for sso, hr_data in groups:
        safe_sso = re.sub(r'[<>:"/\\|?*]', '_', str(sso))
        out_path = os.path.join(HR_OUTPUT_DIR, f"HR_{safe_sso}.xlsx")
        with pd.ExcelWriter(out_path, engine='xlsxwriter') as writer:
            hr_data.to_excel(writer, index=False)
        print(f"Đã tạo file: {out_path} với {len(hr_data)} hàng")
    write_hr_time = time.time() - t5

    total_time = time.time() - start_ts

    # 9) Report timings
    timings = {
        'read_s': round(read_time, 3),
        'merge_s': round(merge_time, 3),
        'map_s': round(map_time, 3),
        'enrich_s': round(enrich_time, 3),
        'write_master_s': round(write_master_time, 3),
        'write_hr_s': round(write_hr_time, 3),
        'total_s': round(total_time, 3),
        'rows_hiring': int(len(hiring_df)),
        'rows_final': int(len(final_df)),
    }
    log_and_print(str(timings))

except Exception as e:
    logging.exception("Pipeline failed")
    print(e)


Duplicate key detected in hiring on 'Emp id': 2 rows across 1 keys
Duplicate key detected in lineup on 'Số CMND/CCCD:': 587 rows across 275 keys
Đã tạo file: hr_files\HR_buiminhph.xlsx với 6 hàng
Đã tạo file: hr_files\HR_ho_thanh.xlsx với 1 hàng
Đã tạo file: hr_files\HR_myduyen_ly.xlsx với 6 hàng
{'read_s': 5.149, 'merge_s': 0.008, 'map_s': 0.008, 'enrich_s': 0.001, 'write_master_s': 0.065, 'write_hr_s': 0.152, 'total_s': 5.422, 'rows_hiring': 22, 'rows_final': 22}
